In [1]:
import pandas as pd
from dateutil import parser

In [2]:
nyc17 = pd.read_csv('/Volumes/TRANSCEND/Violence Tracker/NYC/NYPD_Crime_Data_Current_2017.csv')

In [4]:
nyc17.columns

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC',
       'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'BORO_NM',
       'ADDR_PCT_CD', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM',
       'HADEVELOPT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lat_Lon'],
      dtype='object')

In [5]:
nyc17.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,845348933,03/31/2017,23:30:00,NaN,NaN,03/31/2017,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",...,69.0,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1012423.0,171737.0,40.638018,-73.898491,"(40.638018389, -73.898491201)"
1,886921338,03/31/2017,23:25:00,03/31/2017,23:30:00,03/31/2017,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,14.0,NaN,STREET,NaN,NaN,987466.0,215861.0,40.759173,-73.988393,"(40.759172699, -73.988392793)"
2,893265998,03/31/2017,23:15:00,03/31/2017,23:25:00,03/31/2017,105,ROBBERY,394.0,"ROBBERY,LICENSED FOR HIRE VEHICLE",...,42.0,FRONT OF,TAXI (LIVERY LICENSED),NaN,NaN,1010502.0,245411.0,40.840241,-73.905125,"(40.84024096, -73.905125257)"
3,518511851,03/31/2017,23:00:00,03/31/2017,23:10:00,03/31/2017,364,OTHER STATE LAWS (NON PENAL LA,809.0,TAX LAW,...,49.0,INSIDE,GROCERY/BODEGA,NaN,NaN,1023622.0,253318.0,40.861895,-73.857662,"(40.861894559, -73.85766248)"
4,541009476,03/31/2017,22:55:00,03/31/2017,22:59:00,03/31/2017,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,68.0,FRONT OF,STREET,NaN,NaN,977956.0,167273.0,40.625808,-74.022675,"(40.625808217, -74.022675222)"


In [7]:
nyc17['CMPLNT_FR_DT'] = pd.to_datetime(nyc17['CMPLNT_FR_DT'],errors='coerce') 

In [10]:
nyc17.CMPLNT_FR_DT.dt.year.unique() 

array([ 2017.,  1971.,  2016.,  2015.,  2013.,  2012.,  2008.,    nan,
        2010.,  2014.,  2011.,  2009.,  2005.,  2006.,  1998.,  2007.,
        1984.,  2003.,  1996.,  2004.,  1997.,  1983.,  2002.,  2000.,
        1995.,  2001.,  1916.,  1977.,  1999.,  1986.])

In [16]:
nyc17['Year'], nyc17['Month'] = nyc17['CMPLNT_FR_DT'].dt.year, nyc17['CMPLNT_FR_DT'].dt.month

In [18]:
nyc17 = nyc17[nyc17.Year > 2013] 

In [29]:
murder = ['MURDER & NON-NEGL. MANSLAUGHTER']
flag = nyc17.OFNS_DESC.isin(murder)
nyc_count = nyc17[flag].groupby(['Year','Month','OFNS_DESC']).size().reset_index(name='count')

Note that in 2017's dataset, there might be situations that crime report date is in 2017 but occurred date is in 2016, thus we need to add 2016's number back.

In [30]:
nyc_count

,Year,Month,OFNS_DESC,count
0,2016.0,6.0,MURDER & NON-NEGL. MANSLAUGHTER,1
1,2016.0,8.0,MURDER & NON-NEGL. MANSLAUGHTER,1
2,2016.0,10.0,MURDER & NON-NEGL. MANSLAUGHTER,3
3,2016.0,11.0,MURDER & NON-NEGL. MANSLAUGHTER,2
4,2017.0,1.0,MURDER & NON-NEGL. MANSLAUGHTER,19
5,2017.0,2.0,MURDER & NON-NEGL. MANSLAUGHTER,16
6,2017.0,3.0,MURDER & NON-NEGL. MANSLAUGHTER,16


In [31]:
nyc_count.insert(0,'City','New York City')
nyc_count.insert(1,'State','NY')
nyc_count['UCR_Type'] = ''
nyc_count = nyc_count.rename(columns = {'OFNS_DESC':'homicide_type'})

In [32]:
nyc_count17 = nyc_count[4:] 

In [33]:
nyc_count17

,City,State,Year,Month,homicide_type,count,UCR_Type
4,New York City,NY,2017.0,1.0,MURDER & NON-NEGL. MANSLAUGHTER,19,
5,New York City,NY,2017.0,2.0,MURDER & NON-NEGL. MANSLAUGHTER,16,
6,New York City,NY,2017.0,3.0,MURDER & NON-NEGL. MANSLAUGHTER,16,


In [34]:
nyc_count17.to_csv('NYC_NYPD_homicide_counts_2017.csv',index=False,encoding='utf-8') 

In [40]:
'''
We need add those number back to our original homicide count.
'''
addition_homi_for_16 = nyc_count[:4]

In [38]:
addition_homi_for_16.to_csv('addition_homi_for_16.csv',index=False,encoding='utf-8')
